In [4]:
begin
	import Pkg
	Pkg.activate("..")
	Pkg.instantiate()
	
    push!(LOAD_PATH, "$(@__DIR__)/../src")
    
    using NoisyReach
    using QuadGK
    using ControlSystemsBase
    using LinearAlgebra
    using ReachabilityAnalysis
end

  Activating project at `~/NoisyReach.jl`


┌ Warning: The project dependencies or compat requirements have changed since the manifest was last resolved.
│ It is recommended to `Pkg.resolve()` or consider `Pkg.update()` if necessary.
└ @ Pkg.API /Users/tingan/.julia/juliaup/julia-1.10.3+0.aarch64.apple.darwin14/share/julia/stdlib/v1.10/Pkg/src/API.jl:1807


In [5]:
const Ts = 0.01 # Sampling period

0.01

In [6]:
const Dc = 0.005 # sensor-to-actuator delay

0.005

In [7]:
function matrix_integral(A::AbstractMatrix, B::AbstractMatrix, lower_limit::Float64, upper_limit::Float64)
    integrand = s -> exp(A * s) * B
    result, _ = quadgk(integrand, lower_limit, upper_limit)
    return result
end

matrix_integral (generic function with 1 method)

In [8]:
# Example from page 33 of lecture 17 (Ts = 0.01, Dc = 0.005)
# Expected:
# ϕ = [1 0.001; -0.001 0.999]
# Γ₁ = [0; 0.0005]
# Γ₀ = [0; 0.0005]
#
# ϕ_aug = [1 0.001 0; -0.001 0.999 0.0005; 0 0 0]
# Γ_aug = [0 0.0005 1]
# C_aug = [1 0 0]
sys = let
    A = [0 1; -1 -1]
    B = [0; 1]
    C = [1 0]
    D = 0
    ss(A, B, C, D)
end

StateSpace{Continuous, Int64}
A = 
  0   1
 -1  -1
B = 
 0
 1
C = 
 1  0
D = 
 0

Continuous-time state-space model

In [9]:
sys_aug_ = let
    ϕ = ℯ^(Ts * sys.A)
    Γ₁ = matrix_integral(sys.A, sys.B, Dc, Ts)
    Γ₀ = matrix_integral(sys.A, sys.B, 0.0, Ts - Dc)
    ϕ_aug = [ϕ Γ₁; 0 0 0]
    Γ_aug = [Γ₀; I]
    C_aug = [sys.C 0]
    ss(ϕ_aug, Γ_aug, C_aug, sys.D, Ts)
end

StateSpace{Discrete{Float64}, Float64}
A = 
  0.9999501666658346    0.009950000415833335  3.735416747259115e-5
 -0.009950000415833335  0.9900001662500014    0.00496250038981771
  0.0                   0.0                   0.0
B = 
 1.2479166692686635e-5
 0.004987500026015625
 1.0
C = 
 1.0  0.0  0.0
D = 
 0.0

Sample Time: 0.01 (seconds)
Discrete-time state-space model

In [10]:
#sys_aug = c2d(sys, Ts) * delay(Dc, Ts)
sys_aug = c2d(sys, Ts) * thiran(Dc, Ts)

StateSpace{Discrete{Float64}, Float64}
A = 
  0.9999501666658346    0.009950000415833335   4.4296297035802466e-5
 -0.009950000415833335  0.9900001662500014     0.008844444814074077
  0.0                   0.0                   -0.3333333333333333
B = 
 1.6611111388425926e-5
 0.0033166668052777787
 1.0
C = 
 1.0  0.0  0.0
D = 
 0.0

Sample Time: 0.01 (seconds)
Discrete-time state-space model

In [11]:
K = lqr(ControlSystemsBase.Discrete, sys_aug.A, sys_aug.B, I, I)

1×3 Matrix{Float64}:
 0.239321  0.418528  -0.167879

In [12]:
sys = benchmarks[:F1]
const period = 0.02
const Dc1 = 0.005
const Dc2 = 0.015
const errors = [0, 0]

2-element Vector{Int64}:
 0
 0

In [13]:
sys_aug = let
    ϕ = ℯ^(period * sys.A)
    Γ2 = matrix_integral(sys.A, sys.B, Dc2, period)
    Γ1 = matrix_integral(sys.A, sys.B, Dc1, Dc2)
    Γ3 = matrix_integral(sys.A, sys.B, 0.0, Dc1)
    ϕ_aug = [ϕ Γ3; 0 0 0]
    Γ_aug = [Γ1 Γ2; 0 I]
    C_aug = [sys.C 0]
    D_aug = [sys.D 0]
    ss(ϕ_aug, Γ_aug, C_aug, D_aug, Ts)
end
K = lqr(ControlSystemsBase.Discrete, sys_aug.A, sys_aug.B, I, I)

2×3 Matrix{Float64}:
 0.723651  1.09385  0.0995609
 0.307036  0.48328  0.0441294

In [14]:
λ11 = 0.3
λ12 = 0.25
λ21 = 0.1
λ22 = 0.1
K_error = [K[1,:][1]*(1+λ11) K[1,:][2]*(1+λ12) K[1,:][3]; K[2,:][1]*(1+λ21) K[2,:][2]*(1+λ22) K[2,:][3]]

2×3 Matrix{Float64}:
 0.940746  1.36732   0.0995609
 0.337739  0.531608  0.0441294

In [20]:
H = 10
x0 = 1.
u1_0 = 0.
u2_0 = 0.
z0 = [fill(x0, size(sys.A, 1)); u2_0]
u0 = [u1_0; u2_0]
z = Vector{typeof(z0)}(undef, H+1) 
u = Vector{typeof(u0)}(undef, H)
z[1] = z0
u[1] = u0
z[2] = sys_aug.A * z[1] + sys_aug.B * u[1]
z[2][end-length(u2_0)+1:end, :] .= 0
for k in 2:H
    u[k] = K_error * z[k]
    z[k+1] = sys_aug.A * z[k] + sys_aug.B * u[k]
end

11-element Vector{Vector{Float64}}:
 [1.0, 1.0, 0.0]
 [1.13, 1.0, 0.0]
 [1.30132178769642, 1.5683044969288495, 0.9132527840782902]
 [1.5656335117483775, 2.4684816142869472, 1.3135311127136882]
 [1.9736036029040591, 3.7647623226614937, 1.89900455232343]
 [2.5889090412123625, 5.638527105607954, 2.751741265825341]
 [3.5043427224920793, 8.35341806077923, 3.9932923205910447]
 [4.855029778325602, 12.292975566242283, 5.800515622818432]
 [6.8376814762499665, 18.015197553994717, 8.430745823988177]
 [9.738610399951536, 26.33192409962799, 12.25841197281944]
 [13.974493418376728, 38.42434272595301, 17.828316702794073]